In [1]:
!pip install pytorch-lightning

In [2]:
import torch
from PIL import Image, ImageFile
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import VisionDataset
import pytorch_lightning as pl
from torchvision.models import efficientnet_b0
import torchvision.transforms as T
import os
import pandas as pd
ImageFile.LOAD_TRUNCATED_IMAGES = True

class Regressor(pl.LightningModule):
    def __init__(self, lr=1e-3):
        super().__init__()
        self.backbone = efficientnet_b0(pretrained=True)
        self.backbone.classifier[1] = torch.nn.Linear(in_features=1280, out_features=1, bias=True)
        self.lr = 2e-4

    def forward(self, x):
        # use forward for inference/predictions
        embedding = self.backbone(x)
        return embedding

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("train_loss", loss/1000, on_epoch=True)
        self.log("train_MAE ", torch.sqrt(loss), on_epoch=True)
        return loss/1000

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("valid_loss", loss/1000, on_step=True)
        self.log("val_MAE", torch.sqrt(loss), on_epoch=True)

    def test_step(self, batch, batch_idx):
        x,n = batch
        y_hat = self(x)

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        x, y = batch
        return self(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr,)

In [3]:
class CarData(VisionDataset):
    def __init__(self):
        super().__init__("test_c/")
        self.transform = T.Compose(
            [
                T.Resize((224,224)),
                T.PILToTensor(),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ]
        )
        # self.co2 = pd.read_csv('../datasets/car_models_footprint.csv',sep=";", index_col=0, squeeze=True)["Average of CO2 (g per km)"].to_dict()
        # self.cars = pd.read_csv('../datasets/datasets_train/train_annotation/_annotation.csv', index_col=1, squeeze=True)["models"].to_dict()
        self.images = []
        for img in os.listdir(self.root):
                self.images.append(img)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        with open(os.path.join(self.root,self.images[idx]), "rb") as f:
            img = Image.open(f).convert("RGB")
        return self.transform(img), self.images[idx]

In [7]:
dataset = CarData()
train_size = 4*len(dataset)//5
train, val = random_split(dataset, [train_size,len(dataset)-train_size])
train_loader = DataLoader(train, batch_size=16,num_workers=55, pin_memory=True,shuffle=True)
val_loader = DataLoader(val, batch_size=52,num_workers=55, pin_memory=True)
model = Regressor()
# Initialize a trainer
trainer = pl.Trainer(
    gpus=1,
    max_epochs=100,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [8]:
trainer.fit(model,train_loader,val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type         | Params
------------------------------------------
0 | backbone | EfficientNet | 4.0 M 
------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.035    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [4]:
data_loader = DataLoader(CarData(), batch_size=52,num_workers=55, pin_memory=True)
output = []
model = Regressor().load_from_checkpoint(checkpoint_path="lightning_logs/version_17/checkpoints/epoch=99-step=6599.ckpt")
model.cuda()
model.eval()
i = 0
with torch.no_grad():
    for img, name in data_loader:
        i+=1
        output.append((name,model(img.cuda())))
        print(i)

1
2
3
4
5
6
7
8
9
10
11
12


In [5]:
names = []
co2 = []
for name,c in output:
    names.append(name)
    co2.append(c)

In [11]:
cl = torch.concat(co2,axis=0)[:,0].tolist()

In [6]:
n = []
for ns in names:
    n += ns

In [ ]:
f = []
yt = []
import matplotlib.pyplot as plt
with torch.no_grad():
    for img, y in val_loader:
        f.append(model(img))
        yt.append(y)

In [15]:
import csv
with open('co2_test.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['name','co2'])
    for i in range(len(n)):
        csv_out.writerow((n[i],cl[i]))

In [13]:
len(n)

593

#### 

In [ ]:
f = torch.concat(f)
y = torch.concat(yt)

In [27]:
m = torch.mean(y)
r2 = 1 - torch.sum(torch.square(y-f))/torch.sum(torch.square(y-m))
r2

tensor(0.)

In [28]:
torch.mean(torch.abs(f-y))/m

tensor(0.1938)